# TMDB API로 데이터 수집

## TMDB TV Series & Episode 데이터 수집 (Top 10,000 시리즈 기준)

1. 수집 목적
  - 기존 TMDB TV Series 데이터는 series-level 정보가 불완전하여 결측치가 발생하는 경우가 많음 (예: episode_runtime, directorial credit, writers, detailed cast, season/episode granularity 등)
  - 이를 보완하기 위해 episode-level 데이터를 추가 수집하여 구조·연출·서사 분석이 가능한 고품질 데이터셋을 구축하고자 함
2. 수집 대상: TMDB TV Series(에피소드 포함) 데이터
3. 수집 기간: 2016년 1월 1일 ~ 2025년 11월 29일 (약 11년치)
4. 수집 범위: vote_count 기준 상위 10,000개 에피소드 데이터
  - Series-level : 시리즈 메타데이터는 전수 수집 (Discover 기준 모든 TV Series ID를 수집)
  - Episode-level : 에피소드 데이터는 수집 효율을 고려해 실질적 분석 가치가 높은 시리즈(Top 10,000)에 한해 전체 에피소드 수집 (vote_count 기반 정렬)
    - 사유: 시리즈당 시즌 약 3~8개 × 에피소드 8~24개 → 전체 시리즈(99,000개) × 전체 에피소드 수집 시 약 9,900,000 API 요청 필요 → 3~5일 이상 소요
5. 수집 방식 (비동기 기반 High-performance Pipeline)
  - ① Discover 단계
    - /discover/tv 사용
    - Discover의 500페이지 제한을 회피하기 위해 자동 날짜 분할 방식 적용
  - ② Series 단계 (비동기)
    - TV 시리즈 상세정보 수집
    - /tv/{id} with append_to_response=aggregate_credits, keywords, reviews
    - /tv/{id}/watch/providers 별도 호출
    - rate limiter로 초당 ~30요청 유지
  - ③ Top10k 시리즈 선정
    - series-level 데이터를 정렬하여 vote_count 기반 Top 10,000 추출
  - ④ Episode 단계 (비동기)
    - Top10k 시리즈의 모든 episode 정보 수집/tv/{id}/season/{sn}/episode/{ep}?append_to_response=credits 호출
    - 시리즈-level에는 누락된 감독/작가 정보 보완 (episode_runtime, episode_directors/writers, episode_top_cast 등)
6. 수집 결과: 53개 컬럼, 285415개 데이터, 약 2시간 52분 소요
7. 기타 사항: TV 시리즈 & 에피소드 전수 수집 추가 진행중 (최소 2일 이상 소요 예정)

In [ ]:
# ======================================================
# TMDB TV series 데이터 수집 (A3 NIGHT)
# - 기간: 2016-01-01 ~ 2025-11-29
# - 단계:
#   1) Discover: 전체 TV 시리즈 ID 수집 (자동 날짜 분할)
#   2) Series: 모든 TV 시리즈 상세 수집 (53컬럼)
#   3) Top10k: vote_count 기준 상위 10,000 시리즈 선정
#   4) Episode: Top 10,000 시리즈의 전체 에피소드 + credits(감독/작가/출연) 수집
#
# 특징:
# - 100% 비동기(asyncio + aiohttp) / 멀티스레드 사용 안 함
# - TMDB rate limit 고려 (약 30 req/s)
# - Discover 500 page limit 대비 자동 날짜 분할
# - 시리즈/에피소드 단계별 중단/재개 가능 (done 파일)
# - 최종적으로:
#     tv_series_2016_2025_DISCOVER_v4.csv
#     tv_top10000_by_votecount.csv
#     tv_episodes_top10000_FULL.csv
#   세 개의 핵심 파일 생성
# ======================================================

import os
import time
import json
import csv
from datetime import date, timedelta
from pathlib import Path

import asyncio
import aiohttp
import pandas as pd
from dotenv import load_dotenv

# ------------------------------------------------------
# nest_asyncio: Jupyter에서 async 중복 루프 에러 방지용
# ------------------------------------------------------
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except ImportError:
    print("⚠️ nest_asyncio 없음 (최신 Jupyter는 불필요할 수 있음)")
except Exception as e:
    print(f"⚠️ nest_asyncio 적용 실패: {e}")

# ======================================================
# 0. 기본 설정
# ======================================================
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")
BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}
TIMEOUT = 10           # 각 요청 타임아웃 (초)
MAX_RETRIES = 3        # 재시도 최대 횟수

# Rate limit / 동시성 설정 (비동기용)
RATE_LIMIT_CALLS = 30      # 초당 최대 호출 수 (TMDB 40 미만으로 여유 있게)
RATE_LIMIT_PERIOD = 1.0    # 기준 시간(초)
MAX_CONCURRENT_REQUESTS = 30  # 전체 동시 요청 수 (global semaphore)

# 워커 수
SERIES_WORKERS = 25        # 시리즈 동시 처리 개수
EPISODE_WORKERS = 5        # 에피소드 동시 처리 개수 (안정 우선)

# 수집 기간
GLOBAL_START = date(2016, 1, 1)
GLOBAL_END = date(2025, 11, 29)

# 출력 파일 이름
SERIES_OUT = "tv_series_2016_2025_DISCOVER_v4.csv"
TOP10K_OUT = "tv_top10000_by_votecount.csv"
EPISODE_OUT = "tv_episodes_top10000_FULL.csv"

# 로그 / done 파일
SERIES_DONE_FILE = "series_done.txt"          # 시리즈 상세 수집 완료 ID
SERIES_FAILED_FILE = "series_failed.txt"      # 시리즈 실패 로그
EPISODE_DONE_FILE = "episode_series_done.txt" # 에피소드 수집 완료 시리즈 ID
EPISODE_FAILED_FILE = "episode_series_failed.txt"

# 에피소드 컬럼 정의 (최종 csv 컬럼 순서)
EPISODE_COLUMNS = [
    "series_id","season_number","episode_number","episode_name",
    "episode_air_date","episode_runtime","episode_overview",
    "episode_still_path","episode_vote_average","episode_vote_count",
    "episode_top_cast","episode_directors","episode_writers",
]

# Discover 관련 설정
DISCOVER_PAGE_LIMIT = 500          # TMDB가 허용하는 최대 페이지
DISCOVER_SPLIT_THRESHOLD = 450     # 이 값보다 크면 날짜 범위를 둘로 나눔

# ======================================================
# 1. Rate Limiter + HTTP 공용 함수 (비동기)
# ======================================================
class AsyncRateLimiter:
    """
    초당 요청 수를 제한하기 위한 비동기 Rate Limiter.
    - 최근 'period' 초 동안 'max_calls' 개보다 많이 호출되지 않도록 제어.
    - 429 응답 시, global_backoff_until 시간까지 추가 대기.
    """
    def __init__(self, max_calls=RATE_LIMIT_CALLS, period=RATE_LIMIT_PERIOD):
        self.max_calls = max_calls
        self.period = period
        self.calls = []              # 최근 호출 타임스탬프 기록
        self.global_backoff_until = 0.0
        self._lock = asyncio.Lock()

    async def wait(self):
        """
        요청 전에 호출해서 rate limit를 지키도록 대기하는 메서드.
        """
        async with self._lock:
            now = time.time()

            # 429 혹은 서버 권고로 global backoff가 설정된 경우
            if self.global_backoff_until > now:
                await asyncio.sleep(self.global_backoff_until - now)
                now = time.time()

            # period(1초)보다 오래된 호출 기록 제거
            self.calls = [c for c in self.calls if now - c < self.period]

            # 현재 window에서 너무 많이 호출했으면 대기
            if len(self.calls) >= self.max_calls:
                sleep_time = self.period - (now - self.calls[0]) + 0.01
                if sleep_time > 0:
                    await asyncio.sleep(sleep_time)

            # 현재 호출 시간 기록
            self.calls.append(time.time())

    def set_global_backoff(self, seconds: float):
        """
        429 Retry-After 응답 등을 기반으로 일정 시간 동안 전체 요청을 쉬도록 설정.
        """
        self.global_backoff_until = max(self.global_backoff_until, time.time() + seconds)


# 전역 세션 / 세마포어
rate_limiter = AsyncRateLimiter()
session: aiohttp.ClientSession | None = None
sem = asyncio.BoundedSemaphore(MAX_CONCURRENT_REQUESTS)


async def tmdb_get(path: str, params: dict | None = None, retry_count: int = 0):
    """
    TMDB GET 요청을 수행하는 공용 함수.
    - aiohttp + asyncio 기반 비동기 HTTP 클라이언트
    - 429 / 5xx / Timeout 에 대해 재시도 로직 포함
    - API 키 자동 추가
    """
    if retry_count >= MAX_RETRIES:
        return None
    if API_KEY is None:
        raise RuntimeError("TMDB_API_KEY not found ('.env'에 설정 필요)")

    global session
    if session is None or session.closed:
        session = aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(total=TIMEOUT))

    params = params.copy() if params else {}
    params["api_key"] = API_KEY
    url = f"{BASE_URL}{path}"

    async with sem:
        await rate_limiter.wait()
        try:
            async with session.get(url, params=params, headers=HEADERS) as resp:
                # Rate limit 초과
                if resp.status == 429:
                    retry_after = resp.headers.get("Retry-After")
                    # Retry-After가 너무 길어도 최대 10초까지만 대기
                    wait_sec = min(int(retry_after), 10) if retry_after else min(10 * (2 ** retry_count), 10)
                    print(f"\n🔴 429 Rate Limit: {wait_sec}s wait... ({path})")
                    rate_limiter.set_global_backoff(wait_sec + 2)
                    await asyncio.sleep(wait_sec + 2)
                    return await tmdb_get(path, params, retry_count + 1)

                # 리소스 없음
                if resp.status == 404:
                    return None

                # 서버 에러(5xx) 재시도
                if 500 <= resp.status < 600:
                    if retry_count < MAX_RETRIES:
                        await asyncio.sleep(2 ** retry_count)
                        return await tmdb_get(path, params, retry_count + 1)
                    return None

                resp.raise_for_status()
                return await resp.json()

        except (aiohttp.ClientError, asyncio.TimeoutError):
            # 네트워크 에러 / 타임아웃도 재시도
            if retry_count < MAX_RETRIES:
                await asyncio.sleep(2 ** retry_count)
                return await tmdb_get(path, params, retry_count + 1)
            return None


# ======================================================
# 2. Discover Range Splitter (전체 시리즈 ID 수집)
# ======================================================
async def discover_tv_for_range(start_d: date, end_d: date, depth: int = 0) -> set[int]:
    """
    주어진 날짜 범위 [start_d, end_d]에 대해 /discover/tv를 호출하여
    TV 시리즈 ID를 모두 수집.
    - total_pages > DISCOVER_SPLIT_THRESHOLD 이면 날짜 범위를 둘로 나누어 재귀 호출
    - TMDB의 500 page limit를 피하기 위해 날짜를 자동으로 잘게 split
    """
    indent = "  " * depth
    params = {
        "language": "en-US",
        "include_adult": "false",
        "include_null_first_air_dates": "false",
        "sort_by": "first_air_date.asc",
        "first_air_date.gte": start_d.isoformat(),
        "first_air_date.lte": end_d.isoformat(),
        "page": 1,
    }

    data = await tmdb_get("/discover/tv", params=params)
    if not data:
        print(f"{indent}⚠️ Discover 실패: {start_d} ~ {end_d}")
        return set()

    total_pages = data.get("total_pages", 1)
    total_results = data.get("total_results", 0)
    print(f"{indent}📅 {start_d} ~ {end_d} → total_pages={total_pages}, total_results={total_results}")

    # 페이지가 너무 많으면 날짜 범위를 반으로 나눠서 재귀적으로 수집
    if total_pages > DISCOVER_SPLIT_THRESHOLD and start_d < end_d:
        mid = start_d + (end_d - start_d) // 2
        left_ids = await discover_tv_for_range(start_d, mid, depth + 1)
        right_ids = await discover_tv_for_range(mid + timedelta(days=1), end_d, depth + 1)
        return left_ids | right_ids

    # 현재 범위에서 첫 페이지 결과 처리
    ids: set[int] = set()
    for item in data.get("results", []):
        if item.get("id"):
            ids.add(int(item["id"]))

    # 나머지 페이지들을 비동기로 수집
    max_page = min(total_pages, DISCOVER_PAGE_LIMIT)
    tasks = []
    for page in range(2, max_page + 1):
        p = params.copy()
        p["page"] = page
        tasks.append(tmdb_get("/discover/tv", p))

    for coro in asyncio.as_completed(tasks):
        d = await coro
        if not d:
            continue
        for item in d.get("results", []):
            if item.get("id"):
                ids.add(int(item["id"]))

    return ids


async def collect_all_series_ids() -> list[int]:
    """
    전체 기간(GLOBAL_START ~ GLOBAL_END)에 대해 Discover를 수행,
    고유 TV 시리즈 id 리스트를 반환.
    """
    print("=" * 80)
    print("📌 Discover 전체 시리즈 ID 수집 시작")
    print("=" * 80)

    allids = await discover_tv_for_range(GLOBAL_START, GLOBAL_END)
    ids_list = sorted(allids)
    print(f"\n✅ Discover 완료: 고유 시리즈 {len(ids_list):,}개\n")
    return ids_list


# ======================================================
# 3. Series-level 상세 + providers + credits
# ======================================================
def safe_json_dumps(x):
    """
    JSON 직렬화 도우미.
    - None -> 빈 문자열
    - 직렬화 실패 시에도 빈 문자열 반환 (에러 방지)
    """
    if x is None:
        return ""
    try:
        return json.dumps(x, ensure_ascii=False)
    except Exception:
        return ""


def flat_created_by(lst):
    """
    created_by 필드를
    - JSON 전체 (created_by)
    - 이름만 콤마 조인("created by")
    두 가지 형태로 반환.
    """
    if not isinstance(lst, list):
        return "", ""
    names = [c["name"] for c in lst if c.get("name")]
    return safe_json_dumps(lst), ", ".join(names)


def extract_keywords(obj):
    """
    /keywords 응답에서 keyword 이름들을 콤마로 합쳐 문자열로 반환.
    """
    if not obj:
        return ""
    arr = obj.get("results", [])
    return ", ".join(sorted({x["name"] for x in arr if x.get("name")}))


def extract_review(obj, max_chars: int = 1000):
    """
    /reviews 응답에서 첫 번째 리뷰 내용을 최대 max_chars까지 잘라서 반환.
    """
    if not obj:
        return ""
    arr = obj.get("results", [])
    if not arr:
        return ""
    t = arr[0].get("content", "").replace("\r\n", " ").replace("\n", " ").strip()
    return t[:max_chars] + ("..." if len(t) > max_chars else "")


def extract_cast_crew(obj):
    """
    aggregate_credits 응답에서
    - top_cast: 상위 4명 배우 이름
    - directors: 감독 이름들
    - writers: 작가/시나리오/스토리 관련 스태프 이름들
    을 추출하여 문자열로 반환.
    """
    if not obj:
        return "", "", ""
    cast = obj.get("cast", [])
    crew = obj.get("crew", [])
    top_cast = ", ".join([c["name"] for c in cast[:4] if c.get("name")])

    directors = set()
    writers = set()
    for c in crew:
        name = c.get("name")
        job = (c.get("job", "") + " " + c.get("known_for_department", ""))
        if not name:
            continue
        if "Direct" in job:
            directors.add(name)
        if any(k in job for k in ["Write", "Script", "Story", "Screenplay"]):
            writers.add(name)

    return top_cast, ", ".join(sorted(directors)), ", ".join(sorted(writers))


def extract_providers(obj):
    """
    /watch/providers 응답에서
    - 국가코드:타입:provider_name
    형태의 문자열을 모두 추출하여 콤마로 합쳐 반환.
    ex) "US:flatrate:Netflix, US:ads:Hulu, KR:flatrate:Netflix"
    """
    if not obj:
        return ""
    results = obj.get("results", {})
    out = set()
    for cc, info in results.items():
        if not isinstance(info, dict):
            continue
        for typ in ["flatrate", "ads", "free", "rent", "buy"]:
            lst = info.get(typ)
            if not lst:
                continue
            items = list(lst.values()) if isinstance(lst, dict) else lst
            for p in items:
                if not isinstance(p, dict):
                    continue
                nm = p.get("provider_name")
                if nm:
                    out.add(f"{cc}:{typ}:{nm}")
    return ", ".join(sorted(out))


async def fetch_series_full(tv_id: int):
    """
    한 개의 TV 시리즈에 대해:
    - /tv/{id}?append_to_response=aggregate_credits,keywords,reviews
    - /tv/{id}/watch/providers
    를 호출해서
    53개 series-level 컬럼 + 강화된 credits/provider 정보를 한 행(row)으로 구성.
    반환값: (row(dict), tv_raw_json(dict))
    """
    tv = await tmdb_get(
        f"/tv/{tv_id}",
        params={
            "language": "en-US",
            "append_to_response": "aggregate_credits,keywords,reviews",
        },
    )
    if not tv:
        return None

    agg = tv.get("aggregate_credits") or {}
    kw = tv.get("keywords") or {}
    rv = tv.get("reviews") or {}

    providers = await tmdb_get(f"/tv/{tv_id}/watch/providers")

    cjson, cflat = flat_created_by(tv.get("created_by", []))
    genres = tv.get("genres") or []
    gjson = safe_json_dumps(genres)
    gids = ", ".join(str(g["id"]) for g in genres if g.get("id"))

    # episode_run_time 배열 -> 평균값으로 대표 러닝타임 계산
    ert = tv.get("episode_run_time") or []
    if isinstance(ert, list) and ert:
        try:
            runtime = int(round(sum(ert) / len(ert)))
        except Exception:
            runtime = None
    else:
        runtime = None

    last_ep = tv.get("last_episode_to_air") or {}
    next_ep = tv.get("next_episode_to_air")

    languages = tv.get("languages") or []
    languages_str = ", ".join(languages) if isinstance(languages, list) else str(languages)

    topcast, dirs, wrs = extract_cast_crew(agg)

    row = {
        "id": tv_id,
        "title": tv.get("name"),
        "type": "tv_series",
        "adult": tv.get("adult", False),
        "backdrop_path": tv.get("backdrop_path"),
        "created_by": cjson,
        "episode_run_time": runtime,
        "first_air_date": tv.get("first_air_date"),
        "genres": gjson,
        "genre_ids": gids,
        "homepage": tv.get("homepage"),
        "in_production": tv.get("in_production"),
        "languages": languages_str,
        "last_air_date": tv.get("last_air_date"),

        # last_episode_to_air 전체 필드
        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),

        "name": tv.get("name"),
        "next_episode_to_air": safe_json_dumps(next_ep),
        "networks": safe_json_dumps(tv.get("networks")),
        "number_of_episodes": tv.get("number_of_episodes"),
        "number_of_seasons": tv.get("number_of_seasons"),
        "origin_country": ", ".join(tv.get("origin_country", [])),
        "original_language": tv.get("original_language"),
        "original_name": tv.get("original_name"),
        "overview": tv.get("overview"),
        "popularity": tv.get("popularity"),
        "poster_path": tv.get("poster_path"),
        "production_companies": safe_json_dumps(tv.get("production_companies")),
        "production_countries": safe_json_dumps(tv.get("production_countries")),
        "seasons": safe_json_dumps(tv.get("seasons")),
        "spoken_languages": safe_json_dumps(tv.get("spoken_languages")),
        "status": tv.get("status"),
        "tagline": tv.get("tagline"),
        "type_detail": tv.get("type"),
        "vote_average": tv.get("vote_average"),
        "vote_count": tv.get("vote_count"),

        "review": extract_review(rv),
        "keyword": extract_keywords(kw),
        "top_cast": topcast,
        "directors": dirs,
        "writers": wrs,
        "created by": cflat,
        "providers": extract_providers(providers),
    }
    return row, tv


# ======================================================
# 4. Episode-level 수집 (Top 10,000 시리즈 한정)
# ======================================================
async def fetch_episode(tv_id: int, sn: int, ep_no: int):
    """
    /tv/{tv_id}/season/{sn}/episode/{ep_no}?append_to_response=credits
    를 호출하여 개별 에피소드 정보를 수집.
    - episode 기본 정보
    - credits 기반 top_cast / directors / writers 추출
    """
    d = await tmdb_get(
        f"/tv/{tv_id}/season/{sn}/episode/{ep_no}",
        params={"language": "en-US", "append_to_response": "credits"},
    )
    if not d:
        return None

    cast = d.get("credits", {}).get("cast", [])
    crew = d.get("credits", {}).get("crew", [])

    top = ", ".join([c["name"] for c in cast[:4] if c.get("name")])
    dirs = set()
    wrs = set()
    for c in crew:
        nm = c.get("name")
        job = (c.get("job", "") + " " + c.get("known_for_department", ""))
        if not nm:
            continue
        if "Direct" in job:
            dirs.add(nm)
        if any(k in job for k in ["Write", "Script", "Story", "Screenplay"]):
            wrs.add(nm)

    return {
        "series_id": tv_id,
        "season_number": sn,
        "episode_number": d.get("episode_number"),
        "episode_name": d.get("name"),
        "episode_air_date": d.get("air_date"),
        "episode_runtime": d.get("runtime"),
        "episode_overview": d.get("overview"),
        "episode_still_path": d.get("still_path"),
        "episode_vote_average": d.get("vote_average"),
        "episode_vote_count": d.get("vote_count"),
        "episode_top_cast": top,
        "episode_directors": ", ".join(sorted(dirs)),
        "episode_writers": ", ".join(sorted(wrs)),
    }


async def collect_episodes_for_series(tv_data: dict):
    """
    TMDB TV 상세 응답(tv_data)에서 seasons 정보를 보고
    - 각 시즌의 episode_count만큼
    - 1화부터 N화까지 fetch_episode를 돌려서
    전체 에피소드 정보를 수집.
    """
    if not tv_data:
        return []

    sid = tv_data.get("id")
    seasons = tv_data.get("seasons", [])
    tasks = []

    for s in seasons:
        sn = s.get("season_number")
        ep_c = s.get("episode_count")
        if sn is None or ep_c is None:
            continue
        try:
            sn_int = int(sn)
            ep_count_int = int(ep_c)
        except Exception:
            continue
        if ep_count_int <= 0:
            continue

        for ep in range(1, ep_count_int + 1):
            tasks.append(fetch_episode(sid, sn_int, ep))

    rows = []
    sem_ep = asyncio.Semaphore(EPISODE_WORKERS)

    async def ep_worker(coro):
        async with sem_ep:
            return await coro

    wrapped = [ep_worker(t) for t in tasks]

    for coro in asyncio.as_completed(wrapped):
        r = await coro
        if r:
            rows.append(r)

    return rows


# ======================================================
# 5. done / failed 파일 유틸
# ======================================================
def load_done_set(path: str) -> set[int]:
    """
    각 단계에서 이미 성공적으로 처리된 시리즈 ID를 저장한 파일을 읽어,
    set[int] 형태로 반환.
    """
    if not Path(path).exists():
        return set()
    out: set[int] = set()
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                out.add(int(line))
            except Exception:
                pass
    return out


def append_done_id(path: str, sid: int):
    """
    주어진 sid를 done 파일에 한 줄로 추가 기록.
    """
    with open(path, "a", encoding="utf-8") as f:
        f.write(f"{sid}\n")


def append_failed_id(path: str, sid: int, msg: str):
    """
    실패한 시리즈 ID와 에러 메시지를 탭으로 구분해 로그 파일에 기록.
    """
    with open(path, "a", encoding="utf-8") as f:
        f.write(f"{sid}\t{msg}\n")


# ======================================================
# 6. 시리즈 1개 처리 (series + episode 둘 다에서 사용)
# ======================================================
async def process_one_series_for_series_stage(tv_id: int):
    """
    시리즈 단계에서 사용하는 래퍼:
    - fetch_series_full 호출
    - 실패 시 에러 메시지와 함께 반환
    """
    try:
        res = await fetch_series_full(tv_id)
    except Exception as e:
        return tv_id, None, None, f"{e}"

    if not res:
        return tv_id, None, None, "tv_null"

    row, tv_data = res
    return tv_id, row, tv_data, None


async def process_one_series_for_episode_stage(tv_id: int):
    """
    에피소드 단계에서 사용하는 래퍼:
    - 시리즈 상세(fetch_series_full) 호출
      (이미 series csv가 있어도 episode 수집을 위해 다시 불러옴)
    - collect_episodes_for_series 호출하여 전체 에피소드 수집
    """
    try:
        res = await fetch_series_full(tv_id)
    except Exception as e:
        return tv_id, None, None, f"{e}"

    if not res:
        return tv_id, None, None, "tv_null"

    row, tv_data = res  # row는 여기선 사용하지 않고, tv_data만 사용
    ep_rows = await collect_episodes_for_series(tv_data)
    return tv_id, ep_rows, None, None


# ======================================================
# 7. Main 파이프라인
# ======================================================
async def main():
    """
    전체 파이프라인 실행:
    1) Discover → 전체 시리즈 ID
    2) Series 단계 → 모든 시리즈 상세 수집 + csv 저장
    3) vote_count 기준 상위 10,000 시리즈 선정 → tv_top10000_by_votecount.csv
    4) Episode 단계 → Top 10,000 시리즈 전체 에피소드 수집 + csv 저장
    """
    if API_KEY is None:
        raise RuntimeError("TMDB_API_KEY missing (.env 파일에 TMDB_API_KEY=... 설정 필요)")

    global session

    # --------------------------------------------------
    # STEP 1) Discover - 전체 시리즈 ID
    # --------------------------------------------------
    all_ids = await collect_all_series_ids()

    # --------------------------------------------------
    # STEP 2) Series-level 상세 수집
    # --------------------------------------------------
    print("=" * 80)
    print("📌 STEP 2: TV series 상세 정보 수집 시작")
    print("=" * 80)

    done_series = load_done_set(SERIES_DONE_FILE)
    ids_to_process_series = [sid for sid in all_ids if sid not in done_series]

    print(f"✅ 이미 완료된 시리즈(Series 단계): {len(done_series):,}개")
    print(f"📌 이번에 처리할 시리즈(Series 단계): {len(ids_to_process_series):,}개\n")

    write_header_s = not Path(SERIES_OUT).exists()
    total_start = time.time()
    failed_series = 0
    processed_series = 0
    total_series = len(ids_to_process_series)

    sem_series = asyncio.Semaphore(SERIES_WORKERS)

    async def series_worker(sid: int):
        async with sem_series:
            return await process_one_series_for_series_stage(sid)

    series_tasks = [asyncio.create_task(series_worker(sid)) for sid in ids_to_process_series]

    try:
        for coro in asyncio.as_completed(series_tasks):
            sid, srow, tv_data, err = await coro

            processed_series += 1
            # 진행률 로그
            if processed_series % 20 == 0 or processed_series == 1:
                elapsed = time.time() - total_start
                rate = processed_series / (elapsed / 60) if elapsed > 0 else 0
                remain = (total_series - processed_series) / rate if rate > 0 else 0
                print(
                    f"\r[Series] 진행: {processed_series}/{total_series} "
                    f"({processed_series / total_series * 100:.1f}%) | "
                    f"속도 {rate:.1f}/분 | 남음 {remain:.1f}분 | 실패 {failed_series}",
                    end="",
                    flush=True,
                )

            if err or not srow:
                msg = err or "unknown"
                append_failed_id(SERIES_FAILED_FILE, sid, msg[:200])
                append_done_id(SERIES_DONE_FILE, sid)  # 실패도 done으로 처리 (재시도 원하면 파일 지우고)
                failed_series += 1
                continue

            # Series CSV에 append 저장
            pd.DataFrame([srow]).to_csv(
                SERIES_OUT,
                mode="a",
                index=False,
                encoding="utf-8-sig",
                header=write_header_s,
                quoting=csv.QUOTE_NONNUMERIC,
            )
            write_header_s = False

            append_done_id(SERIES_DONE_FILE, sid)

    except KeyboardInterrupt:
        print("\n⏹ [Series] KeyboardInterrupt! 현재까지 저장된 내용 유지")
    finally:
        # 세션은 에피소드 단계에서도 계속 사용하므로 여기서 닫지 않음
        pass

    elapsed_series = time.time() - total_start
    print(f"\n\n⏱  [Series] 소요시간: {elapsed_series / 3600:.2f}시간")
    print(f"✅ [Series] 처리 완료: {processed_series:,}개, 실패: {failed_series}개\n")

    # ---------------- 시리즈 중복 제거 + 컬럼 정렬 ----------------
    if Path(SERIES_OUT).exists():
        print("📊 [Series] 중복 제거 및 컬럼 정렬 중...")
        series_df = pd.read_csv(SERIES_OUT)

        series_cols_order = [
            "id","title","type","adult","backdrop_path","created_by",
            "episode_run_time","first_air_date","genres","genre_ids",
            "homepage","in_production","languages","last_air_date",
            "last_episode_to_air_id","last_episode_to_air_name",
            "last_episode_to_air_overview","last_episode_to_air_vote_average",
            "last_episode_to_air_vote_count","last_episode_to_air_air_date",
            "last_episode_to_air_episode_number","last_episode_to_air_production_code",
            "last_episode_to_air_runtime","last_episode_to_air_season_number",
            "last_episode_to_air_show_id","last_episode_to_air_still_path",
            "name","next_episode_to_air","networks","number_of_episodes",
            "number_of_seasons","origin_country","original_language",
            "original_name","overview","popularity","poster_path",
            "production_companies","production_countries","seasons",
            "spoken_languages","status","tagline","type_detail",
            "vote_average","vote_count","review","keyword",
            "top_cast","directors","writers","created by","providers",
        ]

        # 누락 컬럼 있으면 추가
        for col in series_cols_order:
            if col not in series_df.columns:
                series_df[col] = None

        series_df = series_df[series_cols_order]

        before = len(series_df)
        series_df = series_df.drop_duplicates(subset=["id"], keep="first")
        after = len(series_df)
        removed = before - after

        if removed > 0:
            print(f"⚠️ [Series] 중복 제거: {removed:,}행 제거")

        series_df.to_csv(
            SERIES_OUT,
            index=False,
            encoding="utf-8-sig",
            quoting=csv.QUOTE_NONNUMERIC,
        )
        print(f"💾 [Series] 최종 저장: {SERIES_OUT} (총 {after:,}개)")

    # --------------------------------------------------
    # STEP 3) vote_count 기준 Top 10,000 시리즈 선정
    # --------------------------------------------------
    print("\n" + "=" * 80)
    print("📌 STEP 3: vote_count 기준 Top 10,000 시리즈 선정")
    print("=" * 80)

    if not Path(SERIES_OUT).exists():
        raise RuntimeError(f"{SERIES_OUT} 파일이 존재하지 않습니다. Series 단계가 끝났는지 확인하세요.")

    series_df = pd.read_csv(SERIES_OUT)

    # vote_count 결측 → 0으로 대체, 숫자형 변환
    series_df["vote_count"] = pd.to_numeric(series_df["vote_count"], errors="coerce").fillna(0).astype(int)

    # 0보다 큰 것들만 대상 (필요하면 0도 포함 가능)
    valid = series_df[series_df["vote_count"] > 0].copy()

    # 정렬 기준: vote_count 내림차순, tie-breaker로 popularity/vote_average도 추가 가능
    valid["popularity"] = pd.to_numeric(valid["popularity"], errors="coerce").fillna(0.0)
    valid["vote_average"] = pd.to_numeric(valid["vote_average"], errors="coerce").fillna(0.0)

    valid = valid.sort_values(
        by=["vote_count", "popularity", "vote_average"],
        ascending=[False, False, False],
    )

    top10k = valid.head(10000).copy()
    top10k[["id", "title", "vote_count", "popularity", "vote_average"]].to_csv(
        TOP10K_OUT,
        index=False,
        encoding="utf-8-sig",
        quoting=csv.QUOTE_NONNUMERIC,
    )

    top_ids = top10k["id"].astype(int).tolist()
    print(f"✅ Top 10,000 시리즈 선정 완료 → {TOP10K_OUT}")

    # --------------------------------------------------
    # STEP 4) Episode 단계 - Top 10,000 전체 에피소드 수집
    # --------------------------------------------------
    print("\n" + "=" * 80)
    print("📌 STEP 4: Top 10,000 시리즈 전체 에피소드 수집 시작")
    print("=" * 80)

    done_episode = load_done_set(EPISODE_DONE_FILE)
    ids_to_process_ep = [sid for sid in top_ids if sid not in done_episode]

    print(f"✅ 이미 완료된 시리즈(Episode 단계): {len(done_episode):,}개")
    print(f"📌 이번에 처리할 시리즈(Episode 단계): {len(ids_to_process_ep):,}개\n")

    write_header_e = not Path(EPISODE_OUT).exists()

    total_start_ep = time.time()
    failed_ep_series = 0
    processed_ep_series = 0
    total_ep_series = len(ids_to_process_ep)

    async def ep_series_worker(sid: int):
        # Top 10,000 시리즈 각각에 대해 episode 전부 수집
        return await process_one_series_for_episode_stage(sid)

    ep_tasks = [asyncio.create_task(ep_series_worker(sid)) for sid in ids_to_process_ep]

    try:
        for coro in asyncio.as_completed(ep_tasks):
            sid, ep_rows, _, err = await coro

            processed_ep_series += 1
            if processed_ep_series % 10 == 0 or processed_ep_series == 1:
                elapsed = time.time() - total_start_ep
                rate = processed_ep_series / (elapsed / 60) if elapsed > 0 else 0
                remain = (total_ep_series - processed_ep_series) / rate if rate > 0 else 0
                print(
                    f"\r[Episode] 진행: {processed_ep_series}/{total_ep_series} "
                    f"({processed_ep_series / total_ep_series * 100:.1f}%) | "
                    f"속도 {rate:.1f}/분 | 남음 {remain:.1f}분 | 실패 {failed_ep_series}",
                    end="",
                    flush=True,
                )

            if err or ep_rows is None:
                msg = err or "unknown"
                append_failed_id(EPISODE_FAILED_FILE, sid, msg[:200])
                append_done_id(EPISODE_DONE_FILE, sid)
                failed_ep_series += 1
                continue

            if ep_rows:
                # 지정한 EPISODE_COLUMNS 순서에 맞춰 저장
                ep_df = pd.DataFrame(ep_rows).reindex(columns=EPISODE_COLUMNS)
                ep_df.to_csv(
                    EPISODE_OUT,
                    mode="a",
                    index=False,
                    encoding="utf-8-sig",
                    header=write_header_e,
                )
                write_header_e = False

            append_done_id(EPISODE_DONE_FILE, sid)

    except KeyboardInterrupt:
        print("\n⏹ [Episode] KeyboardInterrupt! 현재까지 저장된 내용 유지")
    finally:
        if session and not session.closed:
            await session.close()

    elapsed_ep = time.time() - total_start_ep
    print(f"\n\n⏱  [Episode] 소요시간: {elapsed_ep / 3600:.2f}시간")
    print(f"✅ [Episode] 시리즈 단위 처리 완료: {processed_ep_series:,}개, 실패: {failed_ep_series}개\n")

    # ----------- 에피소드 중복 제거 + 컬럼 순서 정렬 -----------
    if Path(EPISODE_OUT).exists():
        print("📊 [Episode] 중복 제거 및 컬럼 순서 정렬 중...")
        ep_df_all = pd.read_csv(EPISODE_OUT)
        before_ep = len(ep_df_all)
        ep_df_all = ep_df_all.drop_duplicates(
            subset=["series_id", "season_number", "episode_number"],
            keep="first",
        )
        after_ep = len(ep_df_all)
        removed_ep = before_ep - after_ep

        if removed_ep > 0:
            print(f"⚠️ [Episode] 중복 제거: {removed_ep:,}행 제거")

        ep_df_all = ep_df_all.reindex(columns=EPISODE_COLUMNS)
        ep_df_all.to_csv(EPISODE_OUT, index=False, encoding="utf-8-sig")
        print(f"💾 [Episode] 최종 저장: {EPISODE_OUT} (총 {after_ep:,}개)")

    print("\n" + "=" * 80)
    print("🎉 전체 파이프라인 완료!")
    print("=" * 80)
    print(f"📌 Series CSV:      {SERIES_OUT}")
    print(f"📌 Top10k 리스트:    {TOP10K_OUT}")
    print(f"📌 Episode CSV:     {EPISODE_OUT}")
    print(f"📌 Series done_set: {SERIES_DONE_FILE}")
    print(f"📌 Series 실패 로그:{SERIES_FAILED_FILE}")
    print(f"📌 Episode done_set:{EPISODE_DONE_FILE}")
    print(f"📌 Episode 실패 로그:{EPISODE_FAILED_FILE}")
    print("=" * 80)


✅ nest_asyncio 적용 완료


In [2]:
await main()

📌 Discover 전체 시리즈 ID 수집 시작
📅 2016-01-01 ~ 2025-11-29 → total_pages=4967, total_results=99328
  📅 2016-01-01 ~ 2020-12-15 → total_pages=1945, total_results=38889
    📅 2016-01-01 ~ 2018-06-24 → total_pages=855, total_results=17089
      📅 2016-01-01 ~ 2017-03-28 → total_pages=410, total_results=8194
      📅 2017-03-29 ~ 2018-06-24 → total_pages=445, total_results=8895
    📅 2018-06-25 ~ 2020-12-15 → total_pages=1090, total_results=21800
      📅 2018-06-25 ~ 2019-09-20 → total_pages=500, total_results=9991
        📅 2018-06-25 ~ 2019-02-06 → total_pages=252, total_results=5022
        📅 2019-02-07 ~ 2019-09-20 → total_pages=249, total_results=4969
      📅 2019-09-21 ~ 2020-12-15 → total_pages=591, total_results=11809
        📅 2019-09-21 ~ 2020-05-03 → total_pages=304, total_results=6079
        📅 2020-05-04 ~ 2020-12-15 → total_pages=287, total_results=5730
  📅 2020-12-16 ~ 2025-11-29 → total_pages=3022, total_results=60439
    📅 2020-12-16 ~ 2023-06-08 → total_pages=1443, total_results

/var/folders/xr/h3sspt_54ygby4y5666yc_380000gn/T/ipykernel_26301/750079805.py:933: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  ep_df_all = pd.read_csv(EPISODE_OUT)


💾 [Episode] 최종 저장: tv_episodes_top10000_FULL.csv (총 285,415개)

🎉 전체 파이프라인 완료!
📌 Series CSV:      tv_series_2016_2025_DISCOVER_v4.csv
📌 Top10k 리스트:    tv_top10000_by_votecount.csv
📌 Episode CSV:     tv_episodes_top10000_FULL.csv
📌 Series done_set: series_done.txt
📌 Series 실패 로그:series_failed.txt
📌 Episode done_set:episode_series_done.txt
📌 Episode 실패 로그:episode_series_failed.txt
